Recoding a GaussianNB from scratch

![](https://images.unsplash.com/photo-1530639834082-05bafb67fbbe?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=1050&q=80)

## Objectives

In this notebook, we will recode the Gaussian Naive Bayes algorithm **from scratch**. <br>

## Guidelines

A quick reminder on **Naive Bayes classification** :
1. The NB classifier is based on the **Bayes Theorem** :

$$ Pr(Y=y_k | X=x) = \frac{Pr(X=x | Y-y_k)Pr(Y=y_k)}{Pr(X=x)}$$

In the above formula :
- $Pr(Y=y_k|X=x)$ is **the probability of each class given the known features**
- $Pr(X=x|Y=y_k)$ is **the probability of the features given the class**
- $Pr(Y=y_k)$ is **the probability of apparition of the class on the given dataset**
- $Pr(X=x)$ **does not actually need to be computed**

Using the Bayes formula, **the algorithm computes the probability of each class for a given set of features, and returns the class with the higest computed probability**.

2. $Pr(Y=y_k)$, the probability of apparition of the class on the given dataset, is very easy to compute : it is just **the percentage of apparition on each class on the training set**.

3. $Pr(X=x|Y=y_k)$, the probability of the features given the class, is a little bit harder to compute, but no stress, let's go step by step :
- The probability of all features given the class is equal to the product of **the probability of each single feature given the class**.
- And **the probability of each single feature given the class can be computed using the following (gaussian) formula** :

$$ P(x_i \mid Y) = \frac{1}{\sqrt{2\pi\sigma^2_Y}} \exp\left(-\frac{(x_i - \mu_Y)^2}{2\sigma^2_Y}\right) $$

In the above formula :
- $\mu_k$ is the mean of the feature $x$ for a given class $y_k$
- $\sigma_k$ is the standard deviation of the feature $x$ for a given class $y_k$

⚠️ **The values of $\mu$, $\sigma$, and the probability of apparition on each class over the training dataset is what the algorithm learns during the fitting stage.**

### 1. From scratch implementation

#### 1.1. Creating the `GaussianNaiveBayes` class

In [1]:
import math
import numpy as np

In [2]:
class GaussianNaiveBayes():
    
    def _get_target_proba(self, y, target):
        """
        Returns the probability of apparition of a given target on a given dataset,
        e.g. the percentage of representation of the given target over the dataset.
        
        Parameters
            y {np.ndarray} -- the target array (1-dimensional)
            target {int} -- the given target
            
        Returns
            float : the proportion of representation of the given target over the dataset.
        """
        proba = y[y == target].size / y.size
        return proba
    
    
    def _get_mu(self, X, y, target):
        """
        Returns the mean of each feature of a dataset for all observations corresponding to a given target.
        
        Parameters
            X {np.ndarray} -- the features array (n-dimensional)
            y {np.ndarray} -- the target array (1-dimensional)
            target {int} -- the given target
            
        Returns
            np.ndarray (1-dim) : the mean value of each feature in the dataset for the given target.
        """
        mu = X[y == target].mean(axis = 0)
        return mu
    
    
    def _get_sigma(self, X, y, target):
        """
        Returns the standard deviation of each feature of a dataset for all observations corresponding to a given target.
        
        Parameters
            X {np.ndarray} -- the features array (n-dimensional)
            y {np.ndarray} -- the target array (1-dimensional)
            target {int} -- the given target
            
        Returns
            np.ndarray (1-dim) : the standard deviation of each feature in the dataset for the given target.
        """
        sigma = X[y == target].std(axis = 0)
        return sigma 
    
    
    def fit(self, X, y):
        """
        Fits the algorithm, e.g. storing as attributes :
            > self.target_probas -- the probability of apparition of each target
            > self.mus -- the mean of each feature for each target
            > self.sigma -- the standard deviation of each feature for each target
        """
        self.target_probas = np.array([self._get_target_proba(y, target) for target in list(set(y))])
        self.mus = np.array([self._get_mu(X, y, target) for target in list(set(y))])
        self.sigmas = np.array([self._get_sigma(X, y, target) for target in list(set(y))])
        
        
    def _get_single_feature_probability(self, x, mu, sigma):
        """
        Returns the probability of apparition of a single feature, given the target.
        Uses the Gaussian probabilistic law.
        
        Parameters
            x {float} -- a single point from a single feature
            mu {float} -- the mean of the feature from which x comes
            sigma {float} -- the standard deviation of the feature from which x comes
            
        Returns
            float : the probability of a single feature point, for a given target.
        """
        feature_proba = 1 / (math.sqrt(2 * math.pi)) * math.e**(-(x-mu)**2/(2*sigma**2))
        return feature_proba
    
    
    def _predict_single_x(self, x):
        """
        Returns the probabilities of belonging to each of the target class for a given data point.
        
        Parameters
            x {np.ndarray} -- a single observation from the dataset (1-dimensional)
            
        Returns
            np.ndarray (1-dimensional) : the array of probabilities of each class for a single point.
        """
        single_proba = np.prod(self._get_single_feature_probability(x, self.mus, self.sigmas), axis = 1) * self.target_probas
        return single_proba
    
    
    def predict_proba(self, X):
        """
        Returns the probabilities of belonging to each of the target class for all points of the dataset.
        
        Parameters
            X {np.ndarray} -- the dataset (2-dimensional)
            
        Returns 
            np.ndarray (2-dimensional) : the array of probabilities of each class for each point in the dataset.
        """
        predict_probas = np.array([self._predict_single_x(x) for x in X])
        return predict_probas

    
    def predict(self, X):
        """
        Returns the predicted class for all points of the dataset.
        
        Parameters
            X {np.ndarray} -- the dataset (2-dimensional)
            
        Returns 
            np.ndarray (1-dimensional) : the predicted class for each point in the dataset.
        """
        pred_proba = self.predict_proba(X)
        pred_class = np.array([np.argmax(pred_proba[i]) for i in range(X.shape[0])])
        
        return pred_class
    
    def score(self, X, y):
        """
        Returns the accuracy score for the model on the dataset X, y.
        
        Parameters
            X {np.ndarray} -- the dataset (2-dimensional)
            y {np.ndarray} -- the target (1-dimensional)
        
        Returns
            float : accuracy score of the model.
        """
        pass

#### 1.2. Testing your model on the iris dataset

In [3]:
from sklearn.datasets import load_iris

iris_df = load_iris()

In [4]:
X = iris_df.data

In [5]:
y = iris_df.target

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

In [7]:
gnb = GaussianNaiveBayes()

In [8]:
gnb.fit(X_train, y_train)

In [9]:
gnb.predict_proba(X_test)

array([[4.31517245e-100, 1.88620105e-004, 1.00968505e-004],
       [4.14903905e-141, 9.92423656e-006, 1.06334270e-003],
       [2.07325099e-106, 4.49417601e-004, 4.33418654e-004],
       [6.96925037e-003, 1.54697093e-019, 1.13275093e-025],
       [3.46789709e-130, 5.64866570e-005, 2.64850486e-003],
       [1.71154769e-037, 2.21362748e-005, 1.09389726e-011],
       [1.16802207e-096, 8.20501523e-004, 1.77421963e-004],
       [7.22712431e-003, 3.80752648e-019, 4.44982720e-025],
       [2.39349168e-267, 6.81487337e-019, 4.87790708e-006],
       [2.21387696e-003, 4.36311495e-020, 5.48789768e-026],
       [7.46738490e-003, 2.75053092e-019, 3.01721083e-025],
       [6.12479256e-176, 5.54608556e-009, 6.82207798e-003],
       [3.48966897e-004, 1.37247559e-017, 2.35156894e-023],
       [5.26901409e-152, 3.50776032e-007, 4.53727163e-003],
       [2.14650760e-003, 2.42302080e-016, 6.58196233e-023],
       [1.76998679e-114, 9.65217686e-004, 1.85985751e-004],
       [3.06474753e-095, 3.39997590e-003

In [10]:
y_pred = gnb.predict(X_test)

In [11]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9666666666666667

### 2. Testing the sklearn version of the GaussianNB on the iris dataset

In [12]:
from sklearn.naive_bayes import GaussianNB
gnb2 = GaussianNB()
gnb2.fit(X_train, y_train)
y_pred = gnb2.predict(X_test)
accuracy_score(y_test, y_pred)

0.9666666666666667

Results are the same between the two models.